In [1]:
import deeplabcut 
import os
from glob import glob
from tqdm import trange, tqdm
from tkinter import filedialog
import tkinter as tk

## DLC config path for the project to retrain

In [2]:
config_path = r'T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\config.yaml'

## Add new videos and label frames, if they are not yet labeled
## Skip this section if the videos are already labeled

In [ ]:
# directory_of_new_unlabeled_vids = ''

In [ ]:
# new_vids = glob(directory_of_new_unlabeled_vids)
# deeplabcut.add_new_videos(config_path,new_vids,copy_videos=True)
# deeplabcut.extract_frames(config_path, mode='automatic', algo='uniform', userfeedback=False, crop=False)

In [ ]:
# deeplabcut.label_frames(config_path)

## Merge the new labaled videos into the project
## Then, create a new training dataset

In [3]:
deeplabcut.merge_datasets(config_path)

The following folder was not manually refined,... T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\labeled-data\092221_PV154black_n_Wsw_2_TOP1
The following folder was not manually refined,... T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\labeled-data\092221_PV154black_n_Wsw_2_TOP2
The following folder was not manually refined,... T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\labeled-data\092221_PV154pink_y_Wsw_2_TOP1
The following folder was not manually refined,... T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\labeled-data\092221_PV154pink_y_Wsw_2_TOP2
The following folder was not manually refined,... T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\labeled-data\092221_PVCha5Agreen_y_Wsw_3_TOP2
The following folder was not manually refined,... T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\labeled-data\100121_PVCha3white_y_Wno_5_TOP1
Please label, or remove the un-corrected folders.


In [4]:
deeplabcut.create_training_dataset(config_path, augmenter_type='imgaug')

T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\training-datasets\iteration-1\UnaugmentedDataSet_BinocularOptoAug12  already exists!
T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\labeled-data\092221_PV154pink_y_Wsw_2_TOP1\CollectedData_Elliott.h5  not found (perhaps not annotated).
T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\labeled-data\092221_PV154pink_y_Wsw_2_TOP2\CollectedData_Elliott.h5  not found (perhaps not annotated).
T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\labeled-data\092221_PV154black_n_Wsw_2_TOP1\CollectedData_Elliott.h5  not found (perhaps not annotated).
T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\labeled-data\092221_PV154black_n_Wsw_2_TOP2\CollectedData_Elliott.h5  not found (perhaps not annotated).
T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\labeled-data\092221_PVCha5Agreen_y_Wsw_3_TOP2\CollectedData_Elliott.h5  not found (perhaps not annotated).
The training dataset is succes

[(0.95,
  1,
  (array([681, 429, 527,  38, 436, 335, 366, 320, 102, 477,  95, 544, 314,
          330, 674, 443, 499,   7,  96,  66, 602, 362, 658,  49, 233, 498,
          302, 390, 446, 194, 133, 246, 698, 626, 305, 169, 315, 495, 711,
          141,  52, 507, 470, 704, 588, 209, 339,  69,  21,  14, 115, 523,
          204, 143, 550, 644, 411,  89, 645, 324, 177, 709, 176, 163, 139,
          422, 218,  68, 158, 650, 683, 225, 433, 471, 373, 247, 534,  74,
          414, 359, 583, 224, 138, 134, 123, 548,  44,  36, 654, 447, 520,
          297, 181, 316, 485, 323, 258, 212, 168, 619, 132, 622, 623, 453,
          404,  12, 279, 322, 574,  28,  43, 539, 231, 291,  33, 461, 536,
          377, 671, 392, 592, 331, 455, 130, 283, 606, 463, 581, 666, 487,
          652, 277, 427, 355, 348, 113,  26, 472, 474, 191, 569, 172, 265,
          395, 280,  51, 308, 689, 688, 234, 525, 244, 594, 665, 341, 668,
          557,  54, 458, 551, 437, 170, 502, 467, 399, 387, 184, 564, 222,
          43

## Set up training options and rewrite config.yaml

In [5]:
# this should be where training ended the last time this network was trained
existing_snapshot = r'T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\dlc-models\iteration-0\BinocularOptoAug12-trainset95shuffle1\train\snapshot-275000'
starting_iters = 275000

In [6]:
# batch size (I use =14 w/ 3090, =8 or =10 is good on a 2080ti)
batchsize = 10

In [7]:
cfg = deeplabcut.auxiliaryfunctions.read_config(config_path)
trainposeconfigfile, testposeconfigfile, snapshotfolder = deeplabcut.return_train_network_path(config_path)
cfg_dlc = deeplabcut.auxiliaryfunctions.read_plainconfig(trainposeconfigfile)
cfg_dlc['scale_jitter_lo'] = 0.5
cfg_dlc['scale_jitter_up'] = 1.5
cfg_dlc['augmentationprobability'] = 0.5
cfg_dlc['batch_size'] = batchsize
cfg_dlc['elastic_transform'] = True
cfg_dlc['rotation'] = True # make this False if it's an eyecam dataset, True is good if it's topdown
cfg_dlc['covering'] = True
cfg_dlc['motion_blur'] = True
cfg_dlc['optimizer'] = "adam"
cfg_dlc['dataset_type'] = 'imgaug' # 'imagaug'multi-animal-
cfg_dlc['multi_step'] = [[1e-4, int(75000/batchsize)+starting_iters], [5.0e-5, int(120000/batchsize)+starting_iters], [1e-5, int(500000/batchsize)+starting_iters]]
cfg_dlc['init_weights'] = existing_snapshot
deeplabcut.auxiliaryfunctions.write_plainconfig(trainposeconfigfile, cfg_dlc)

## Train

In [8]:
deeplabcut.train_network(config_path, allow_growth=True, displayiters=500, maxiters=starting_iters+100000, saveiters=10000)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15]],
 'all_joints_names': ['Nose',
                      'Lear',
                      'Rear',
                      'BackofHead',
                      'Spine',
                      'TailBase',
                      'Cricket1',
                      'Cricket2',
                      'TopRight1',
                      'TopRight2',
                      'TopLeft1',
                      'TopLeft2',
                      'BotLeft1',
                      'BotLeft2',
                      'BotRight1',
                      'BotRight2'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'augmentationprobability': 0.5,
 'batch_size': 10,
 'clahe': True,
 'clahe

Selecting single-animal trainer
Batch Size is 10


C:\ProgramData\Anaconda3\envs\ephys1-gpu\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Loading already trained DLC with backbone: resnet_50
Max_iters overwritten as 325000
Display_iters overwritten as 500
Save_iters overwritten as 10000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'T:\\deeplabcut_projects\\BinocularOpto-Elliott-2021-08-12-new\\dlc-models\\iteration-1\\BinocularOptoAug12-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'adam', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 10, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False

iteration: 275500 loss: 0.0030 lr: 0.0001
iteration: 276000 loss: 0.0026 lr: 0.0001
iteration: 276500 loss: 0.0024 lr: 0.0001
iteration: 277000 loss: 0.0023 lr: 0.0001
iteration: 277500 loss: 0.0022 lr: 0.0001
iteration: 278000 loss: 0.0021 lr: 0.0001
iteration: 278500 loss: 0.0021 lr: 0.0001
iteration: 279000 loss: 0.0020 lr: 0.0001
iteration: 279500 loss: 0.0020 lr: 0.0001
iteration: 280000 loss: 0.0019 lr: 0.0001
iteration: 280500 loss: 0.0019 lr: 0.0001
iteration: 281000 loss: 0.0019 lr: 0.0001
iteration: 281500 loss: 0.0019 lr: 0.0001
iteration: 282000 loss: 0.0018 lr: 0.0001
iteration: 282500 loss: 0.0018 lr: 0.0001
iteration: 283000 loss: 0.0015 lr: 5e-05
iteration: 283500 loss: 0.0015 lr: 5e-05
iteration: 284000 loss: 0.0015 lr: 5e-05
iteration: 284500 loss: 0.0014 lr: 5e-05
iteration: 285000 loss: 0.0014 lr: 5e-05
iteration: 285500 loss: 0.0014 lr: 5e-05
iteration: 286000 loss: 0.0014 lr: 5e-05
iteration: 286500 loss: 0.0014 lr: 5e-05
iteration: 287000 loss: 0.0014 lr: 5e-05
i

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


## Evaluate trained network

In [3]:
deeplabcut.evaluate_network(config_path, plotting=True, trainingsetindex='all')

T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new/evaluation-results/  already exists!
Running  DLC_resnet50_BinocularOptoAug12shuffle1_325000  with # of trainingiterations: 325000


C:\ProgramData\Anaconda3\envs\ephys1-gpu\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Analyzing data...


712it [01:07, 10.53it/s]


Done and results stored for snapshot:  snapshot-325000
Results for 325000  training iterations: 95 1 train error: 2.31 pixels. Test error: 19.38  pixels.
With pcutoff of 0.6  train error: 2.31 pixels. Test error: 18.47 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...


100%|████████████████████████████████████████████████████████████████████████████████| 712/712 [04:06<00:00,  2.89it/s]

The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Evaluate and plot novel video

In [5]:
deeplabcut.analyze_videos(config_path, [r'C:\Users\Niell lab\Documents\temp_preycap_dlc\091621_PV154black_y_Wlb_1_TOP2.avi'])

Using snapshot-325000 for model T:\deeplabcut_projects\BinocularOpto-Elliott-2021-08-12-new\dlc-models\iteration-1\BinocularOptoAug12-trainset95shuffle1


C:\ProgramData\Anaconda3\envs\ephys1-gpu\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Starting to analyze %  C:\Users\Niell lab\Documents\temp_preycap_dlc\091621_PV154black_y_Wlb_1_TOP2.avi
C:\Users\Niell lab\Documents\temp_preycap_dlc  already exists!
Loading  C:\Users\Niell lab\Documents\temp_preycap_dlc\091621_PV154black_y_Wlb_1_TOP2.avi
Duration of video [s]:  39.02 , recorded with  60.0 fps!
Overall # of frames:  2341  found with (before cropping) frame dimensions:  720 540
Starting to extract posture


2346it [00:42, 55.28it/s]

Saving results in C:\Users\Niell lab\Documents\temp_preycap_dlc...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet50_BinocularOptoAug12shuffle1_325000'

In [6]:
deeplabcut.create_labeled_video(config_path, [r'C:\Users\Niell lab\Documents\temp_preycap_dlc\091621_PV154black_y_Wlb_1_TOP2.avi'], save_frames=True, trailpoints=10)